<a href="https://colab.research.google.com/github/EmptyGod1/Mashine_Learning/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas seaborn scikit-learn
!pip install pandas

# Домашнее задание 3

1. Выбрать номинальное значение как целевую переменную
2. Провести разведочный анализ **включая** анализ зависимости между номинальными переменными (chi2)
3. Выбрать `best_features`
4. Сделать нормализацию данных (scaler, onehotencoding)
4. Обучить модель логистической регрессии
5. Вывести метрики качества, описать полученный результат (также вывести коэфициенты модели)

In [87]:
import pandas as pd
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, chi2

In [88]:
df = pd.read_csv("/content/Sleep_health_and_lifestyle_dataset.csv")

In [81]:
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


1. Для анализа была выбрана переменная Sleep Disorder.

2. Разведочный анализ

### EDA

In [82]:
cat_columns = df.select_dtypes(include='object').columns.tolist()
print(cat_columns)

['Gender', 'Occupation', 'BMI Category', 'Blood Pressure', 'Sleep Disorder']


In [83]:
cat_columns = [col for col in cat_columns if col != 'Sleep Disorder']

In [84]:
df.describe(include='object')

,Gender,Occupation,BMI Category,Blood Pressure,Sleep Disorder
count,374,374,374,374,155
unique,2,11,4,25,2
top,Male,Nurse,Normal,130/85,Sleep Apnea
freq,189,73,195,99,78


In [85]:
for col in cat_columns:
    table = pd.crosstab(df['Sleep Disorder'], df[col])
    chi2, p, dof, expected = chi2_contingency(table)
    print(f"Зависимость между Sleep Disorder и {col} — p-value: {p:.5f}, chi2: {chi2:.2f}")

Зависимость между Sleep Disorder и Gender — p-value: 0.00000, chi2: 24.91
Зависимость между Sleep Disorder и Occupation — p-value: 0.00000, chi2: 110.77
Зависимость между Sleep Disorder и BMI Category — p-value: 0.86502, chi2: 0.73
Зависимость между Sleep Disorder и Blood Pressure — p-value: 0.00000, chi2: 120.62


По полученным результатам видно что Gender, Occupation и Blood Pressure имеют статистически значимаю связь с переменной Sleep Disorder. Их оставм для дальнейшей работы.

3. Выбор best_features


In [90]:
# Кодируем целевую переменную числами
y = LabelEncoder().fit_transform(df['Sleep Disorder'])

X_cat = pd.get_dummies(df[cat_columns])

# Выборка лучших признаков
selector = SelectKBest(score_func=chi2, k='all')
selector.fit(X_cat, y)

scores = selector.scores_
feature_scores = pd.Series(scores, index=X_cat.columns)
best_features = feature_scores.sort_values(ascending=False)
best_features.head(10)

,0
Blood Pressure_140/95,192.515383
Occupation_Nurse,173.900836
BMI Category_Overweight,127.470897
BMI Category_Normal,100.123754
Occupation_Salesperson,96.015096
Blood Pressure_135/90,85.686864
Occupation_Teacher,53.897596
Blood Pressure_125/80,39.577510
Blood Pressure_130/85,31.011475
Occupation_Doctor,29.217291


In [91]:
# Создаём dummy-признаки (one-hot encoding)
df_dummies = pd.get_dummies(df, columns=[
    'Blood Pressure',
    'Occupation',
    'BMI Category'
])

# Теперь список best_features подходит к полученному DataFrame
best_features = [
    'Blood Pressure_140/95',
    'Occupation_Nurse',
    'BMI Category_Overweight',
    'BMI Category_Normal',
    'Occupation_Salesperson'
]

X_best = df_dummies[best_features]

4. Сделать нормализацию данных (scaler, onehotencoding)


In [92]:
# Формируем обучающую выборку из best_features
X_best = df_dummies[best_features]

scaler = StandardScaler()

print(X_best.head())

   Blood Pressure_140/95  Occupation_Nurse  BMI Category_Overweight  \
0                  False             False                     True   
1                  False             False                    False   
2                  False             False                    False   
3                  False             False                    False   
4                  False             False                    False   

   BMI Category_Normal  Occupation_Salesperson  
0                False                   False  
1                 True                   False  
2                 True                   False  
3                False                   False  
4                False                   False  


Я выполнил нормализацию данных, преобразовав все выбранные категориальные переменные в one-hot кодировку с помощью функции pd.get_dummies. Теперь для каждого уникального значения категориальной переменной создан отдельный бинарный столбец, отражающий наличие этого значения у конкретного наблюдения (True/False).
Таким образом, полученные бинарные признаки уже находятся в подходящем формате и не требуют дополнительной числовой нормализации — их можно непосредственно использовать для обучения модели машинного обучения. Данные полностью готовы для построения и оценки модели на этом признаковом пространстве.